## Import Supply Chain Graph Data into Neo4j Aura


This notebook connects to a **Neo4j Aura** instance and imports graph data (nodes and relationships) from a JSON file generated by a Cypher path export.
Make sure to:
- Upload your `supplychaindata.json` file
- Replace the Neo4j URI and credentials below with your Aura instance details


In [ ]:

# Neo4j Connection Setup
from neo4j import GraphDatabase

from dotenv import load_dotenv
import os
import json 
load_dotenv('ws-data.env', override=True)

# Replace these with your Neo4j credentials and connection URI
# Neo4j
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')

# Create a Neo4j driver instance
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

In [ ]:

# Load JSON data
with open("data/supplychaindata.json", "r") as f:
    data = json.load(f)


In [ ]:

# Helper function to create nodes and relationships
def import_path(tx, segment):
    start = segment["start"]
    end = segment["end"]
    rel = segment["relationship"]

    # Build MERGE statements for nodes and relationship
    query = (
        f"MERGE (a:{':'.join(start['labels'])} {{id: $start_id}})\n"
        f"SET a += $start_props\n"
        f"MERGE (b:{':'.join(end['labels'])} {{id: $end_id}})\n"
        f"SET b += $end_props\n"
        f"MERGE (a)-[r:{rel['type']}]->(b)\n"
        f"SET r += $rel_props"
    )

    tx.run(query,
           start_id=start["elementId"],
           start_props=start["properties"],
           end_id=end["elementId"],
           end_props=end["properties"],
           rel_props=rel["properties"])


In [ ]:

# Upload data to Neo4j
with driver.session() as session:
    for record in data:
        for segment in record["p"]["segments"]:
            session.write_transaction(import_path, segment)

print("✅ Data imported successfully into Neo4j Aura.")
